In [1]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

cuda:0


In [2]:
from transformers import BartTokenizer, BartModel

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large') # param 400M, file size 1G 
model = BartModel.from_pretrained('facebook/bart-large')
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
# outputs = model(**inputs)
# last_hidden_states = outputs.last_hidden_state
# print(last_hidden_states)

/home/seelur/enter/envs/pytorch_2_1_0/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/seelur/enter/envs/pytorch_2_1_0/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
print(type(inputs))
print(inputs.keys())
print(inputs["input_ids"].shape)
print(inputs["attention_mask"].shape)
print(*inputs) # keays


<class 'transformers.tokenization_utils_base.BatchEncoding'>
dict_keys(['input_ids', 'attention_mask'])
torch.Size([1, 8])
torch.Size([1, 8])
input_ids attention_mask


In [4]:
model.to(device) # 使用GPU
input_ids = inputs["input_ids"].to(device)
attention_mask = inputs["attention_mask"].to(device)

In [5]:

outputs = model(input_ids, attention_mask) # 所以模型的输入是 input_ids 和 attention_mask
last_hidden_states = outputs.last_hidden_state
print(last_hidden_states)

Inside BART: Encoder: input_ids= tensor([[    0, 31414,     6,   127,  2335,    16, 11962,     2]],
       device='cuda:0')
Inside BART: input_ids= tensor([[    2,     0, 31414,     6,   127,  2335,    16, 11962]],
       device='cuda:0')
Inside BART: attention_mask= None
Inside BART: encoder_hidden_states= tensor([[[-0.0089,  0.0095,  0.0101,  ...,  0.0054, -0.0061, -0.0019],
         [-0.0242, -0.2558,  0.1059,  ..., -0.1760,  0.1102,  0.0296],
         [ 0.0278, -0.2268,  0.0643,  ...,  0.0303,  0.1538, -0.1770],
         ...,
         [-0.0684, -0.1958, -0.0589,  ..., -0.0166,  0.1663, -0.1595],
         [-0.0161, -0.3884, -0.4422,  ..., -0.0780,  0.0474, -0.0166],
         [ 0.0912,  0.0469, -0.0327,  ..., -0.0290, -0.0870,  0.2132]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward0>)
Inside BART: encoder_attention_mask= tensor([[1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')
Inside BART: head_mask= None
Inside BART: cross_attn_head_mask= None
Inside BART: past_key_values= N

In [6]:
print(outputs['last_hidden_state'].shape)
model.device

torch.Size([1, 8, 1024])


device(type='cuda', index=0)

In [7]:
# # Visualization # 模型是ONNX格式的，不能直接可视化
# from torch.utils.tensorboard import SummaryWriter
# writer = SummaryWriter('./BART')

# # add_graph() will trace the sample input through your model,
# # and render it as a graph.
# # writer.add_graph(model, input_ids, attention_mask)
# writer.add_graph(model, inputs['input_ids'], inputs['attention_mask'])

# writer.flush()

# # To view, start TensorBoard on the command line with:
# #   tensorboard --logdir=runs
# # ...and open a browser tab to http://localhost:6006/
# # 注意上面的'runs'是日志保存的路径，在前面的代码中有设置

In [8]:
model

BartModel(
  (shared): Embedding(50265, 1024, padding_idx=1)
  (encoder): BartEncoder(
    (embed_tokens): Embedding(50265, 1024, padding_idx=1)
    (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
    (layers): ModuleList(
      (0-11): 12 x BartEncoderLayer(
        (self_attn): BartAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (activation_fn): GELUActivation()
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      )
    )
    (layerno

In [9]:
model.decoder

BartDecoder(
  (embed_tokens): Embedding(50265, 1024, padding_idx=1)
  (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
  (layers): ModuleList(
    (0-11): 12 x BartDecoderLayer(
      (self_attn): BartAttention(
        (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
      )
      (activation_fn): GELUActivation()
      (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (encoder_attn): BartAttention(
        (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
      )
      (e

In [10]:
# 只使用embedder进行运算，得到encoder的运算结果 
embeddings = model.shared(input_ids) # 所以模型的输入是 input_ids 和 attention_mask
print(embeddings.shape)

torch.Size([1, 8, 1024])


In [11]:
# 只使用encoder进行运算，得到encoder的运算结果，注意调用encoder的时候它默认调佣 embedder
encoder_outputs = model.encoder(input_ids, attention_mask) # 所以模型的输入是 input_ids 和 attention_mask
representations = encoder_outputs.last_hidden_state
print(encoder_outputs.keys())
print(representations.shape)
print(representations)

Inside BART: Encoder: input_ids= tensor([[    0, 31414,     6,   127,  2335,    16, 11962,     2]],
       device='cuda:0')
odict_keys(['last_hidden_state'])
torch.Size([1, 8, 1024])
tensor([[[-0.0089,  0.0095,  0.0101,  ...,  0.0054, -0.0061, -0.0019],
         [-0.0242, -0.2558,  0.1059,  ..., -0.1760,  0.1102,  0.0296],
         [ 0.0278, -0.2268,  0.0643,  ...,  0.0303,  0.1538, -0.1770],
         ...,
         [-0.0684, -0.1958, -0.0589,  ..., -0.0166,  0.1663, -0.1595],
         [-0.0161, -0.3884, -0.4422,  ..., -0.0780,  0.0474, -0.0166],
         [ 0.0912,  0.0469, -0.0327,  ..., -0.0290, -0.0870,  0.2132]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward0>)


In [12]:
# 只使用decoder进行运算，得到decoder的运算结果，注意调用decoder的时候它默认调佣 embedder
intermediate_output = model.decoder(input_ids, attention_mask) # 所以模型的输入是 input_ids 和 attention_mask
predictions = intermediate_output.last_hidden_state
print(intermediate_output.keys())
print(predictions.shape)
print(type(intermediate_output['past_key_values']), len(intermediate_output['past_key_values'])) # 这个应该是指这次保存的 key 和 value值，可以用来做 window attention
print(type(intermediate_output['past_key_values'][0]), len(intermediate_output['past_key_values'][0]))
print(type(intermediate_output['past_key_values'][0][0]), len(intermediate_output['past_key_values'][0][0].shape))
print(predictions) # 这里输出的结果和运行完整模型的结果不一致

Inside BART: input_ids= tensor([[    0, 31414,     6,   127,  2335,    16, 11962,     2]],
       device='cuda:0')
Inside BART: attention_mask= tensor([[1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')
Inside BART: encoder_hidden_states= None
Inside BART: encoder_attention_mask= None
Inside BART: head_mask= None
Inside BART: cross_attn_head_mask= None
Inside BART: past_key_values= None
Inside BART: inputs_embeds= None
Inside BART: use_cache= None
Inside BART: output_attentions= None
Inside BART: output_hidden_states= None
Inside BART: return_dict= None
odict_keys(['last_hidden_state', 'past_key_values'])
torch.Size([1, 8, 1024])
<class 'tuple'> 12
<class 'tuple'> 2
<class 'torch.Tensor'> 4
tensor([[[-0.7063,  1.2007,  0.1487,  ...,  0.4761, -0.7794, -1.1263],
         [-0.7752,  1.0035,  0.8857,  ...,  1.0058,  0.2381, -1.0515],
         [-0.2852, -1.1637,  1.8671,  ..., -0.6132,  0.3621, -1.3947],
         ...,
         [-1.4383,  0.5772,  2.3473,  ...,  0.7001,  0.2143,  1.4135],
         

In [13]:
# 现在的问题是，怎么将 encoder 的输出 单独输入给 decoder？
# 是否可以直接给模型的参数赋值？

In [14]:
# representations torch.Size([1, 8, 1024])
print(type(model.decoder.layers), len(model.decoder.layers)) # <class 'torch.nn.modules.container.ModuleList'>  12
model.decoder.layers[0]

<class 'torch.nn.modules.container.ModuleList'> 12


BartDecoderLayer(
  (self_attn): BartAttention(
    (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
    (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
    (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
    (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
  )
  (activation_fn): GELUActivation()
  (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (encoder_attn): BartAttention(
    (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
    (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
    (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
    (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
  )
  (encoder_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (fc1): Linear(in_features=1024, out_features=4096, bias=True)
  (fc2): Linear(in_features=4096, out_features=1024, bias=True)
  (final_layer_norm): Laye

In [15]:
# 只使用decoder进行运算，得到decoder的运算结果，注意调用decoder的时候它默认调佣 embedder
temp_ids = torch.tensor([[    2,     0, 31414,     6,   127,  2335,    16, 11962]]).to(device)
intermediate_output = model.decoder(temp_ids, 
                                    encoder_hidden_states=encoder_outputs[0], 
                                    encoder_attention_mask=attention_mask,
                                    use_cache=True,
                                    return_dict=True) # 所以模型的输入是 input_ids 和 attention_mask
predictions = intermediate_output.last_hidden_state
print(intermediate_output.keys())
print(predictions.shape)
print(type(intermediate_output['past_key_values']), len(intermediate_output['past_key_values'])) # 这个应该是指这次保存的 key 和 value值，可以用来做 window attention
print(type(intermediate_output['past_key_values'][0]), len(intermediate_output['past_key_values'][0]))
print(type(intermediate_output['past_key_values'][0][0]), len(intermediate_output['past_key_values'][0][0].shape))
print(predictions) # 这里输出的结果和运行完整模型的结果不一致

'''
OK，通过打印在两种使用情况下 Decoder接收到的输入，发现，一个关键点，那就是 在EncoderDecoder架构中，
Decoder接收到的input_ids是被shuffle过的。为什么？发生在什么时候？
破案了，当直接使用EncoderDecoder的时候，如果没有传入 decoder_input_ids， 那么就会通过对 input_ids 调用 shift_tokens_right 函数
得到decoder_input_ids。该函数实现向右移动一个token的功能。

'''



Inside BART: input_ids= tensor([[    2,     0, 31414,     6,   127,  2335,    16, 11962]],
       device='cuda:0')
Inside BART: attention_mask= None
Inside BART: encoder_hidden_states= tensor([[[-0.0089,  0.0095,  0.0101,  ...,  0.0054, -0.0061, -0.0019],
         [-0.0242, -0.2558,  0.1059,  ..., -0.1760,  0.1102,  0.0296],
         [ 0.0278, -0.2268,  0.0643,  ...,  0.0303,  0.1538, -0.1770],
         ...,
         [-0.0684, -0.1958, -0.0589,  ..., -0.0166,  0.1663, -0.1595],
         [-0.0161, -0.3884, -0.4422,  ..., -0.0780,  0.0474, -0.0166],
         [ 0.0912,  0.0469, -0.0327,  ..., -0.0290, -0.0870,  0.2132]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward0>)
Inside BART: encoder_attention_mask= tensor([[1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')
Inside BART: head_mask= None
Inside BART: cross_attn_head_mask= None
Inside BART: past_key_values= None
Inside BART: inputs_embeds= None
Inside BART: use_cache= True
Inside BART: output_attentions= None
Inside BART: output_h

'\nOK，通过打印在两种使用情况下 Decoder接收到的输入，发现，一个关键点，那就是 在EncoderDecoder架构中，\nDecoder接收到的input_ids是被shuffle过的。为什么？\n'

In [16]:
print(encoder_outputs[0].shape)

torch.Size([1, 8, 1024])


{'input_ids': [[101, 1045, 1005, 102]], 'attention_mask': [[1, 1, 1, 1]]}
